# **License Plate Detection**

In [3]:
#!pip install easyocr
#!pip install ultralytics

In [4]:
import ast
import cv2 as cv
import easyocr
from glob import glob
import numpy as np
import pandas as pd
import string
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


- ## Making use of regular pre-trained yolov8 model for car recognition
- ## And Our Custom trained license detection model

In [5]:
coco_model = YOLO('yolov8s.pt')

np_model = YOLO('/content/drive/MyDrive/Colab Notebooks/license_plate/license_trn.pt')

100%|██████████| 21.5M/21.5M [00:00<00:00, 445MB/s]


In [6]:
# read in test video paths
videos = glob('/content/drive/MyDrive/Colab Notebooks/license_plate/media/sample.mp4')
print(videos)

['/content/drive/MyDrive/Colab Notebooks/license_plate/media/sample.mp4']


# Training Our Model (with default function)

In [7]:
# read video by index
video = cv.VideoCapture(videos[0])

ret = True
frame_number = -1
# all vehicle class IDs from the COCO dataset (car, motorbike, truck) 
vehicles = [2,3,5]
vehicle_bounding_boxes = []

# read the 10 first frames
while ret:
    frame_number += 1
    ret, frame = video.read()

    if ret and frame_number < 10:
        # use track() to identify instances and track them frame by frame
        detections = coco_model.track(frame, persist=True)[0]
        # save cropped detections
        # detections.save_crop('outputs')
        # print nodel predictions for debugging
        # print(results)

        for detection in detections.boxes.data.tolist():
            # print detection bounding boxes for debugging
            # print(detection)
            x1, y1, x2, y2, track_id, score, class_id = detection
            # I am only interested in class IDs that belong to vehicles
            if int(class_id) in vehicles and score > 0.5:
                vehicle_bounding_boxes.append([x1, y1, x2, y2, track_id, score])

# print found bounding boxes for debugging
print(vehicle_bounding_boxes)
video.release()

requirements: Ultralytics requirement ['lap>=0.5.12'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 118.0 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 2.5s, installed 1 package: ['lap>=0.5.12']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect


0: 384x640 19 cars, 6 trucks, 38.5ms
Speed: 14.8ms preprocess, 38.5ms inference, 754.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 19 cars, 6 trucks, 10.9ms
Speed: 3.8ms preprocess, 10.9ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 18 cars, 7 trucks, 10.9ms
Speed: 3.7ms preprocess, 10.9ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 19 cars, 6 trucks, 11.1ms
Speed: 3.5ms preprocess, 11.1ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 19 cars, 6 trucks, 10.9ms
Speed: 3.5ms preprocess, 10.9ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 64

In [8]:
# read video by index
video = cv.VideoCapture(videos[0])

ret = True
frame_number = -1
vehicles = [2,3,5]

# read the 10 first frames
while ret:
    frame_number += 1
    ret, frame = video.read()

    if ret and frame_number < 10:

        # vehicle detector
        detections = coco_model.track(frame, persist=True)[0]
        for detection in detections.boxes.data.tolist():
            x1, y1, x2, y2, track_id, score, class_id = detection
            if int(class_id) in vehicles and score > 0.5:
                vehicle_bounding_boxes = []
                vehicle_bounding_boxes.append([x1, y1, x2, y2, track_id, score])
                for bbox in vehicle_bounding_boxes:
                    print(bbox)
                    roi = frame[int(y1):int(y2), int(x1):int(x2)]
                    # debugging check if bbox lines up with detected vehicles (should be identical to save_crops() above
                    # cv.imwrite(str(track_id) + '.jpg', roi)

                    # license plate detector for region of interest
                    license_plates = np_model(roi)[0]
                    # check every bounding box for a license plate
                    for license_plate in license_plates.boxes.data.tolist():
                        plate_x1, plate_y1, plate_x2, plate_y2, plate_score, _ = license_plate
                        # verify detections
                        print(license_plate, 'track_id: ' + str(bbox[4]))
                        plate = roi[int(plate_y1):int(plate_y2), int(plate_x1):int(plate_x2)]
                        cv.imwrite(str(track_id) + '.jpg', plate)

video.release()


0: 384x640 19 cars, 6 trucks, 16.8ms
Speed: 3.9ms preprocess, 16.8ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)
[2172.234619140625, 1158.6678466796875, 2806.598876953125, 1740.30712890625, 1.0, 0.8923707604408264]

0: 608x640 1 license_plate, 57.9ms
Speed: 4.3ms preprocess, 57.9ms inference, 2.4ms postprocess per image at shape (1, 3, 608, 640)
[248.42686462402344, 409.18902587890625, 412.8212585449219, 474.27667236328125, 0.7154296040534973, 0.0] track_id: 1.0
[19.385900497436523, 684.3803100585938, 380.45947265625, 970.2046508789062, 2.0, 0.8888245820999146]

0: 512x640 (no detections), 58.0ms
Speed: 3.3ms preprocess, 58.0ms inference, 0.8ms postprocess per image at shape (1, 3, 512, 640)
[752.01513671875, 1371.3209228515625, 1451.3277587890625, 1988.8330078125, 3.0, 0.8852115273475647]

0: 576x640 1 license_plate, 54.0ms
Speed: 4.4ms preprocess, 54.0ms inference, 1.7ms postprocess per image at shape (1, 3, 576, 640)
[241.4958953857422, 412.7590026855469, 438.0

In [9]:
# read video by index
video = cv.VideoCapture(videos[0])

ret = True
frame_number = -1
vehicles = [2,3,5]

# read the 10 first frames
while ret:
    frame_number += 1
    ret, frame = video.read()

    if ret and frame_number < 100:

        # vehicle detector
        detections = coco_model.track(frame, persist=True)[0]
        for detection in detections.boxes.data.tolist():
            x1, y1, x2, y2, track_id, score, class_id = detection
            if int(class_id) in vehicles and score > 0.5:
                vehicle_bounding_boxes = []
                vehicle_bounding_boxes.append([x1, y1, x2, y2, track_id, score])
                for bbox in vehicle_bounding_boxes:
                    print(bbox)
                    roi = frame[int(y1):int(y2), int(x1):int(x2)]

                    # license plate detector for region of interest
                    license_plates = np_model(roi)[0]
                    # process license plate
                    for license_plate in license_plates.boxes.data.tolist():
                        plate_x1, plate_y1, plate_x2, plate_y2, plate_score, _ = license_plate
                        # crop plate from region of interest
                        plate = roi[int(plate_y1):int(plate_y2), int(plate_x1):int(plate_x2)]
                        # de-colorize
                        plate_gray = cv.cvtColor(plate, cv.COLOR_BGR2GRAY)
                        # posterize
                        _, plate_treshold = cv.threshold(plate_gray, 64, 255, cv.THRESH_BINARY_INV)
                        cv.imwrite(str(track_id) + '_gray.jpg', plate_gray)
                        cv.imwrite(str(track_id) + '_thresh.jpg', plate_treshold)

video.release()

Streaming output truncated to the last 5000 lines.
Speed: 3.3ms preprocess, 10.8ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)
[2174.94287109375, 1180.49658203125, 2791.314453125, 1766.5372314453125, 1.0, 0.8945235013961792]

0: 608x640 1 license_plate, 8.3ms
Speed: 4.0ms preprocess, 8.3ms inference, 1.2ms postprocess per image at shape (1, 3, 608, 640)
[13.775568962097168, 688.5045166015625, 386.25445556640625, 970.5054321289062, 2.0, 0.8597905039787292]

0: 512x640 (no detections), 7.8ms
Speed: 3.1ms preprocess, 7.8ms inference, 0.5ms postprocess per image at shape (1, 3, 512, 640)
[729.0297241210938, 1397.9652099609375, 1442.336181640625, 2030.4149169921875, 3.0, 0.8941800594329834]

0: 576x640 1 license_plate, 11.0ms
Speed: 3.9ms preprocess, 11.0ms inference, 1.6ms postprocess per image at shape (1, 3, 576, 640)
[1136.0697021484375, 1326.96044921875, 1654.71337890625, 1825.346435546875, 4.0, 0.8433287739753723]

0: 640x640 (no detections), 9.7ms
Speed: 3.9ms pr

## Initialize the OCR reader

In [10]:
reader = easyocr.Reader(['en'], gpu=True)

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

## Default Function for reading the license plate text

In [11]:
def read_license_plate(license_plate_crop):
    detections = reader.readtext(license_plate_crop)

    for detection in detections:
        bbox, text, score = detection

        text = text.upper().replace(' ', '')

        return text, score

    return None, None

### Writing and saving our result in a csv file . . .

In [12]:
def write_csv(results, output_path):

    with open(output_path, 'w') as f:
        f.write('{},{},{},{},{},{},{},{}\n'.format(
            'frame_number', 'track_id', 'car_bbox', 'car_bbox_score',
            'license_plate_bbox', 'license_plate_bbox_score', 'license_plate_number',
            'license_text_score'))

        for frame_number in results.keys():
            for track_id in results[frame_number].keys():
                print(results[frame_number][track_id])
                if 'car' in results[frame_number][track_id].keys() and \
                   'license_plate' in results[frame_number][track_id].keys() and \
                   'number' in results[frame_number][track_id]['license_plate'].keys():
                    f.write('{},{},{},{},{},{},{},{}\n'.format(
                        frame_number,
                        track_id,
                        '[{} {} {} {}]'.format(
                            results[frame_number][track_id]['car']['bbox'][0],
                            results[frame_number][track_id]['car']['bbox'][1],
                            results[frame_number][track_id]['car']['bbox'][2],
                            results[frame_number][track_id]['car']['bbox'][3]
                        ),
                        results[frame_number][track_id]['car']['bbox_score'],
                        '[{} {} {} {}]'.format(
                            results[frame_number][track_id]['license_plate']['bbox'][0],
                            results[frame_number][track_id]['license_plate']['bbox'][1],
                            results[frame_number][track_id]['license_plate']['bbox'][2],
                            results[frame_number][track_id]['license_plate']['bbox'][3]
                        ),
                        results[frame_number][track_id]['license_plate']['bbox_score'],
                        results[frame_number][track_id]['license_plate']['number'],
                        results[frame_number][track_id]['license_plate']['text_score'])
                    )
        f.close()

In [13]:
results = {}

# read video by index
video = cv.VideoCapture(videos[0])

ret = True
frame_number = -1
vehicles = [2,3,5]

# read the 10 first frames
while ret:
    frame_number += 1
    ret, frame = video.read()

    if ret and frame_number < 100:
        results[frame_number] = {}

        # vehicle detector
        detections = coco_model.track(frame, persist=True)[0]
        for detection in detections.boxes.data.tolist():
            x1, y1, x2, y2, track_id, score, class_id = detection
            if int(class_id) in vehicles and score > 0.5:
                vehicle_bounding_boxes = []
                vehicle_bounding_boxes.append([x1, y1, x2, y2, track_id, score])
                for bbox in vehicle_bounding_boxes:
                    print(bbox)
                    roi = frame[int(y1):int(y2), int(x1):int(x2)]

                    # license plate detector for region of interest
                    license_plates = np_model(roi)[0]
                    # process license plate
                    for license_plate in license_plates.boxes.data.tolist():
                        plate_x1, plate_y1, plate_x2, plate_y2, plate_score, _ = license_plate
                        # crop plate from region of interest
                        plate = roi[int(plate_y1):int(plate_y2), int(plate_x1):int(plate_x2)]
                        # de-colorize
                        plate_gray = cv.cvtColor(plate, cv.COLOR_BGR2GRAY)
                        # posterize
                        _, plate_treshold = cv.threshold(plate_gray, 64, 255, cv.THRESH_BINARY_INV)

                        # OCR
                        np_text, np_score = read_license_plate(plate_treshold)
                        # if plate could be read write results
                        if np_text is not None:
                            results[frame_number][track_id] = {
                                'car': {
                                    'bbox': [x1, y1, x2, y2],
                                    'bbox_score': score
                                },
                                'license_plate': {
                                    'bbox': [plate_x1, plate_y1, plate_x2, plate_y2],
                                    'bbox_score': plate_score,
                                    'number': np_text,
                                    'text_score': np_score
                                }
                            }

write_csv(results, '/content/drive/MyDrive/Colab Notebooks/license_plate/results.csv')
video.release()

Streaming output truncated to the last 5000 lines.
0: 384x640 20 cars, 7 trucks, 10.3ms
Speed: 3.9ms preprocess, 10.3ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)
[715.867919921875, 1415.9737548828125, 1425.6353759765625, 2057.185302734375, 4.0, 0.8984678387641907]

0: 608x640 1 license_plate, 10.3ms
Speed: 4.0ms preprocess, 10.3ms inference, 1.5ms postprocess per image at shape (1, 3, 608, 640)
[2037.1646728515625, 884.1861572265625, 2439.67529296875, 1238.121337890625, 5.0, 0.862525224685669]

0: 576x640 (no detections), 14.4ms
Speed: 3.4ms preprocess, 14.4ms inference, 0.8ms postprocess per image at shape (1, 3, 576, 640)
[1128.59521484375, 1353.093505859375, 1647.8935546875, 1846.7138671875, 6.0, 0.890110433101654]

0: 608x640 (no detections), 11.5ms
Speed: 4.1ms preprocess, 11.5ms inference, 0.6ms postprocess per image at shape (1, 3, 608, 640)
[619.4246215820312, 575.7213745117188, 831.0584716796875, 773.0064086914062, 8.0, 0.570999801158905]

0: 608x640 (no

- ### When we inspect the saved csv file(`results.csv`), we observe that the result is very inaccurate with a low 'license_text_score' ...
- ### this is because there a thousands of different liscenses with unique formats ...
- ### as a result, our model is working with a high number of different possibilities which greatly affects the accuracy.

# Creating additional functions to improve the accuracy
- ### Mapping dictionaries for character conversion
- ### Characters that can easily be confused can be verified by their location. Example; an `O` in a place where a number is expected is probably a `0`

In [14]:
dict_char_to_int = {'O': '0',
                    'I': '1',
                    'J': '3',
                    'A': '4',
                    'G': '6',
                    'S': '5'}

dict_int_to_char = {'0': 'O',
                    '1': 'I',
                    '3': 'J',
                    '4': 'A',
                    '6': 'G',
                    '5': 'S'}

- ## I will specifically design the model to detect License Plate of the format `AB12CDE` ...
- ## this is the License plate format of the *United Kingdom* .
- ## This is because of the `sample.mp4` video we are working with.

In [15]:
def license_complies_format(text):
    # True if the license plate complies with the format, False otherwise.
    if len(text) != 7:
        return False

    if (text[0] in string.ascii_uppercase or text[0] in dict_int_to_char.keys()) and \
       (text[1] in string.ascii_uppercase or text[1] in dict_int_to_char.keys()) and \
       (text[2] in ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'] or text[2] in dict_char_to_int.keys()) and \
       (text[3] in ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'] or text[3] in dict_char_to_int.keys()) and \
       (text[4] in string.ascii_uppercase or text[4] in dict_int_to_char.keys()) and \
       (text[5] in string.ascii_uppercase or text[5] in dict_int_to_char.keys()) and \
       (text[6] in string.ascii_uppercase or text[6] in dict_int_to_char.keys()):
        return True
    else:
        return False

In [16]:
def format_license(text):
    license_plate_ = ''
    mapping = {0: dict_int_to_char, 1: dict_int_to_char, 4: dict_int_to_char, 5: dict_int_to_char, 6: dict_int_to_char,
               2: dict_char_to_int, 3: dict_char_to_int}
    for j in [0, 1, 2, 3, 4, 5, 6]:
        if text[j] in mapping[j].keys():
            license_plate_ += mapping[j][text[j]]
        else:
            license_plate_ += text[j]

    return license_plate_

In [17]:
def read_license_plate(license_plate_crop):
    detections = reader.readtext(license_plate_crop)

    for detection in detections:
        bbox, text, score = detection

        text = text.upper().replace(' ', '')

        # verify that text is conform to a standard license plate
        if license_complies_format(text):
            # bring text into the default license plate format
            return format_license(text), score

    return None, None

# Training our model (with specific functions to fit our data)

In [18]:
results = {}

# read video by index
video = cv.VideoCapture(videos[0])

ret = True
frame_number = -1
vehicles = [2,3,5]

# read the entire video
while ret:
    ret, frame = video.read()
    frame_number += 1
    if ret:
        results[frame_number] = {}

        # vehicle detector
        detections = coco_model.track(frame, persist=True)[0]
        for detection in detections.boxes.data.tolist():
            x1, y1, x2, y2, track_id, score, class_id = detection
            if int(class_id) in vehicles and score > 0.5:
                vehicle_bounding_boxes = []
                vehicle_bounding_boxes.append([x1, y1, x2, y2, track_id, score])
                for bbox in vehicle_bounding_boxes:
                    print(bbox)
                    roi = frame[int(y1):int(y2), int(x1):int(x2)]

                    # license plate detector for region of interest
                    license_plates = np_model(roi)[0]
                    # process license plate
                    for license_plate in license_plates.boxes.data.tolist():
                        plate_x1, plate_y1, plate_x2, plate_y2, plate_score, _ = license_plate
                        # crop plate from region of interest
                        plate = roi[int(plate_y1):int(plate_y2), int(plate_x1):int(plate_x2)]
                        # de-colorize
                        plate_gray = cv.cvtColor(plate, cv.COLOR_BGR2GRAY)
                        # posterize
                        _, plate_treshold = cv.threshold(plate_gray, 64, 255, cv.THRESH_BINARY_INV)

                        # OCR
                        np_text, np_score = read_license_plate(plate_treshold)
                        # if plate could be read write results
                        if np_text is not None:
                            results[frame_number][track_id] = {
                                'car': {
                                    'bbox': [x1, y1, x2, y2],
                                    'bbox_score': score
                                },
                                'license_plate': {
                                    'bbox': [plate_x1, plate_y1, plate_x2, plate_y2],
                                    'bbox_score': plate_score,
                                    'number': np_text,
                                    'text_score': np_score
                                }
                            }

write_csv(results, '/content/drive/MyDrive/Colab Notebooks/license_plate/results1.csv')
video.release()

Streaming output truncated to the last 5000 lines.
0: 288x640 (no detections), 15.7ms
Speed: 4.9ms preprocess, 15.7ms inference, 0.6ms postprocess per image at shape (1, 3, 288, 640)
[2275.66455078125, 1666.18359375, 3165.823974609375, 2150.447998046875, 2217.0, 0.8802050948143005]

0: 352x640 (no detections), 13.0ms
Speed: 3.1ms preprocess, 13.0ms inference, 0.7ms postprocess per image at shape (1, 3, 352, 640)
[2250.13916015625, 1103.2152099609375, 2760.683837890625, 1505.8011474609375, 2482.0, 0.8532347083091736]

0: 512x640 1 license_plate, 12.6ms
Speed: 3.9ms preprocess, 12.6ms inference, 2.0ms postprocess per image at shape (1, 3, 512, 640)
[2102.061279296875, 853.42578125, 2492.3505859375, 1180.26611328125, 2714.0, 0.8121321797370911]

0: 544x640 (no detections), 9.3ms
Speed: 3.3ms preprocess, 9.3ms inference, 0.7ms postprocess per image at shape (1, 3, 544, 640)
[918.0725708007812, 1595.39501953125, 1610.56591796875, 2158.125, 2353.0, 0.8387675881385803]

0: 544x640 (no detecti

In [20]:
results = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/license_plate/results1.csv')

# show results for tracking ID `1` - sort by OCR prediction confidence
results[results['track_id'] == 6].sort_values(by='license_text_score', ascending=False)

,frame_number,track_id,car_bbox,car_bbox_score,license_plate_bbox,license_plate_bbox_score,license_plate_number,license_text_score
25,30,6.0,[688.9998168945312 1464.7767333984375 1408.600...,0.914819,[247.48538208007812 437.6631774902344 454.9360...,0.682333,NA13NRU,0.785712
27,31,6.0,[687.8892822265625 1466.058837890625 1408.1597...,0.914556,[248.3437957763672 435.55633544921875 454.9605...,0.707300,NA13NRU,0.628000
42,42,6.0,[658.8450317382812 1508.96142578125 1394.47473...,0.902099,[254.60977172851562 444.3173522949219 472.7067...,0.664598,NA13NRU,0.580949
13,14,6.0,[719.7803344726562 1404.0181884765625 1437.727...,0.890917,[249.7278594970703 432.50164794921875 448.8240...,0.715016,NA13NRU,0.579887
32,34,6.0,[678.3192138671875 1476.6151123046875 1405.041...,0.911522,[251.43910217285156 441.1811828613281 459.2988...,0.728375,NA13NRU,0.557782
3,3,6.0,[739.9058837890625 1371.6099853515625 1438.991...,0.885648,[246.15110778808594 423.1602478027344 446.9494...,0.701431,NA13NRU,0.555826
16,16,6.0,[715.8679809570312 1415.9737548828125 1425.635...,0.898468,[244.76376342773438 430.50067138671875 442.765...,0.722104,NA13NRU,0.540874
37,39,6.0,[669.1422119140625 1497.2242431640625 1393.524...,0.902610,[252.47828674316406 439.4128112792969 471.7760...,0.620289,NA13NRU,0.490439
11,12,6.0,[722.58056640625 1399.726318359375 1442.746948...,0.901221,[245.97084045410156 431.5975646972656 446.4314...,0.705560,NA13NRU,0.479306
47,46,6.0,[650.34130859375 1517.869140625 1393.032836914...,0.909076,[257.3167724609375 451.47967529296875 470.6333...,0.713594,NA13MRU,0.428909


## We see that the accuracy of our model has greatly increased!

### Adding frames with no detections so that the output will be displayed smoothly ...

In [21]:
import csv
import numpy as np
from scipy.interpolate import interp1d


def interpolate_bounding_boxes(data):
    # Extract necessary data columns from input data
    frame_numbers = np.array([int(row['frame_number']) for row in data])
    car_ids = np.array([int(float(row['track_id'])) for row in data])
    car_bboxes = np.array([list(map(float, row['car_bbox'][1:-1].split())) for row in data])
    license_plate_bboxes = np.array([list(map(float, row['license_plate_bbox'][1:-1].split())) for row in data])

    interpolated_data = []
    unique_car_ids = np.unique(car_ids)
    for car_id in unique_car_ids:

        frame_numbers_ = [p['frame_number'] for p in data if int(float(p['track_id'])) == int(float(car_id))]
        print(frame_numbers_, car_id)

        # Filter data for a specific car ID
        car_mask = car_ids == car_id
        car_frame_numbers = frame_numbers[car_mask]
        car_bboxes_interpolated = []
        license_plate_bboxes_interpolated = []

        first_frame_number = car_frame_numbers[0]
        last_frame_number = car_frame_numbers[-1]

        for i in range(len(car_bboxes[car_mask])):
            frame_number = car_frame_numbers[i]
            car_bbox = car_bboxes[car_mask][i]
            license_plate_bbox = license_plate_bboxes[car_mask][i]

            if i > 0:
                prev_frame_number = car_frame_numbers[i-1]
                prev_car_bbox = car_bboxes_interpolated[-1]
                prev_license_plate_bbox = license_plate_bboxes_interpolated[-1]

                if frame_number - prev_frame_number > 1:
                    # Interpolate missing frames' bounding boxes
                    frames_gap = frame_number - prev_frame_number
                    x = np.array([prev_frame_number, frame_number])
                    x_new = np.linspace(prev_frame_number, frame_number, num=frames_gap, endpoint=False)
                    interp_func = interp1d(x, np.vstack((prev_car_bbox, car_bbox)), axis=0, kind='linear')
                    interpolated_car_bboxes = interp_func(x_new)
                    interp_func = interp1d(x, np.vstack((prev_license_plate_bbox, license_plate_bbox)), axis=0, kind='linear')
                    interpolated_license_plate_bboxes = interp_func(x_new)

                    car_bboxes_interpolated.extend(interpolated_car_bboxes[1:])
                    license_plate_bboxes_interpolated.extend(interpolated_license_plate_bboxes[1:])

            car_bboxes_interpolated.append(car_bbox)
            license_plate_bboxes_interpolated.append(license_plate_bbox)

        for i in range(len(car_bboxes_interpolated)):
            frame_number = first_frame_number + i
            row = {}
            row['frame_number'] = str(frame_number)
            row['track_id'] = str(car_id)
            row['car_bbox'] = ' '.join(map(str, car_bboxes_interpolated[i]))
            row['license_plate_bbox'] = ' '.join(map(str, license_plate_bboxes_interpolated[i]))

            if str(frame_number) not in frame_numbers_:
                # Imputed row, set the following fields to '0'
                row['license_plate_bbox_score'] = '0'
                row['license_plate_number'] = '0'
                row['license_text_score'] = '0'
            else:
                # Original row, retrieve values from the input data if available
                original_row = [p for p in data if int(p['frame_number']) == frame_number and int(float(p['track_id'])) == int(float(car_id))][0]
                row['license_plate_bbox_score'] = original_row['license_plate_bbox_score'] if 'license_plate_bbox_score' in original_row else '0'
                row['license_plate_number'] = original_row['license_plate_number'] if 'license_plate_number' in original_row else '0'
                row['license_text_score'] = original_row['license_text_score'] if 'license_text_score' in original_row else '0'

            interpolated_data.append(row)

    return interpolated_data


# Load the CSV file
with open('/content/drive/MyDrive/Colab Notebooks/license_plate/results1.csv', 'r') as file:
    reader = csv.DictReader(file)
    data = list(reader)

# Interpolate missing data
interpolated_data = interpolate_bounding_boxes(data)

# Write updated data to a new CSV file
header = ['frame_number', 'track_id', 'car_bbox', 'license_plate_bbox', 'license_plate_bbox_score', 'license_plate_number', 'license_text_score']
with open('results1_interpolated.csv', 'w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=header)
    writer.writeheader()
    writer.writerows(interpolated_data)


['148', '149', '152', '153', '154', '155', '156', '157', '158', '159', '160', '161', '162', '163', '164', '165', '166', '167', '168', '169', '170', '174', '175', '183', '188', '189', '190', '191', '192', '193', '196', '202', '203', '204', '205', '206', '208', '209', '210', '211', '212', '213', '214', '215', '218', '219', '220', '221', '222', '223', '224', '225', '226', '231', '232', '233', '234', '235', '236', '237', '238', '239', '240', '241', '242', '243', '244', '245', '246', '247', '248', '249', '250', '251', '252', '253', '254', '255', '256', '257', '258', '259', '260', '261', '262', '263', '264', '265', '266', '268', '269', '270', '271', '272', '273', '274', '275', '276', '277', '278', '279', '280', '281', '282', '283', '284', '285', '286', '287', '288', '289', '290', '291', '292', '293', '294', '295', '296', '297', '298', '299', '300', '301', '302', '303', '304', '305', '306', '307', '308', '309', '310', '311', '312', '313', '314', '315', '316', '317', '318', '319', '320', '321'

In [22]:
def draw_border(img, top_left, bottom_right, color=(0, 255, 0), thickness=6, line_length_x=200, line_length_y=200):
    x1, y1 = top_left
    x2, y2 = bottom_right

    cv.line(img, (x1, y1), (x1, y1 + line_length_y), color, thickness)  #-- top-left
    cv.line(img, (x1, y1), (x1 + line_length_x, y1), color, thickness)

    cv.line(img, (x1, y2), (x1, y2 - line_length_y), color, thickness)  #-- bottom-left
    cv.line(img, (x1, y2), (x1 + line_length_x, y2), color, thickness)

    cv.line(img, (x2, y1), (x2 - line_length_x, y1), color, thickness)  #-- top-right
    cv.line(img, (x2, y1), (x2, y1 + line_length_y), color, thickness)

    cv.line(img, (x2, y2), (x2, y2 - line_length_y), color, thickness)  #-- bottom-right
    cv.line(img, (x2, y2), (x2 - line_length_x, y2), color, thickness)

    return img

# Saving Our Detections and displaying the license plate text

In [24]:
results = pd.read_csv('/content/results1_interpolated.csv')

# load video
video_path = '/content/drive/MyDrive/Colab Notebooks/license_plate/media/sample.mp4'
cap = cv.VideoCapture(video_path)

fourcc = cv.VideoWriter_fourcc(*'mp4v')  # Specify the codec
fps = cap.get(cv.CAP_PROP_FPS)
width = int(cap.get(cv.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv.CAP_PROP_FRAME_HEIGHT))
out = cv.VideoWriter('./out.mp4', fourcc, fps, (width, height))

license_plate = {}
for car_id in np.unique(results['track_id']):
    max_ = np.amax(results[results['track_id'] == car_id]['license_text_score'])
    license_plate[car_id] = {'license_crop': None,
                             'license_plate_number': results[(results['track_id'] == car_id) &
                                                             (results['license_text_score'] == max_)]['license_plate_number'].iloc[0]}
    cap.set(cv.CAP_PROP_POS_FRAMES, results[(results['track_id'] == car_id) &
                                             (results['license_text_score'] == max_)]['frame_number'].iloc[0])
    ret, frame = cap.read()

    x1, y1, x2, y2 = ast.literal_eval(results[(results['track_id'] == car_id) &
                                              (results['license_text_score'] == max_)]['license_plate_bbox'].iloc[0].replace('[ ', '[').replace('   ', ' ').replace('  ', ' ').replace(' ', ','))

    license_crop = frame[int(y1):int(y2), int(x1):int(x2), :]
    license_crop = cv.resize(license_crop, (int((x2 - x1) * 400 / (y2 - y1)), 400))

    license_plate[car_id]['license_crop'] = license_crop


frame_nmr = -1

cap.set(cv.CAP_PROP_POS_FRAMES, 0)

# read frames
ret = True
while ret:
    ret, frame = cap.read()
    frame_nmr += 1
    if ret:
        df_ = results[results['frame_number'] == frame_nmr]
        for row_indx in range(len(df_)):
            # draw car
            car_x1, car_y1, car_x2, car_y2 = ast.literal_eval(df_.iloc[row_indx]['car_bbox'].replace('[ ', '[').replace('   ', ' ').replace('  ', ' ').replace(' ', ','))
            draw_border(frame, (int(car_x1), int(car_y1)), (int(car_x2), int(car_y2)), (0, 255, 0), 25,
                        line_length_x=200, line_length_y=200)

            # draw license plate
            x1, y1, x2, y2 = ast.literal_eval(df_.iloc[row_indx]['license_plate_bbox'].replace('[ ', '[').replace('   ', ' ').replace('  ', ' ').replace(' ', ','))
            cv.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 255), 12)

            # crop license plate
            license_crop = license_plate[df_.iloc[row_indx]['track_id']]['license_crop']

            H, W, _ = license_crop.shape

            try:
                #frame[int(car_y1) - H - 100:int(car_y1) - 100,
                      #int((car_x2 + car_x1 - W) / 2):int((car_x2 + car_x1 + W) / 2), :] = license_crop

                frame[int(car_y1) - H - 400:int(car_y1) - H - 100,
                      int((car_x2 + car_x1 - W) / 2):int((car_x2 + car_x1 + W) / 2), :] = (255, 255, 255)

                (text_width, text_height), _ = cv.getTextSize(
                    license_plate[df_.iloc[row_indx]['track_id']]['license_plate_number'],
                    cv.FONT_HERSHEY_SIMPLEX,
                    4.3,
                    17)

                cv.putText(frame,
                            license_plate[df_.iloc[row_indx]['track_id']]['license_plate_number'],
                            (int((car_x2 + car_x1 - text_width) / 2), int(car_y1 - H - 250 + (text_height / 2))),
                            cv.FONT_HERSHEY_SIMPLEX,
                            4.3,
                            (0, 0, 0),
                            17)

            except:
                pass

        out.write(frame)
        frame = cv.resize(frame, (1280, 720))

        # cv2.imshow('frame', frame)
        # cv2.waitKey(0)

out.release()
cap.release()